In [ ]:
# TensorFlow 1 버전 사용
%tensorflow_version 1.x

In [ ]:
import os

In [ ]:
# 디렉토리 변경
os.chdir('drive/My Drive/Colab Notebooks/DACON/AIFrenz_Season2/')

# 데이터 압축 풀기

In [ ]:
import zipfile

In [ ]:
# train 데이터 압축 해제
train_path = './Data/train'
if not os.path.exists(train_path):
    train_zip = zipfile.ZipFile('./Data/train.zip')
    train_zip.extractall(train_path)
    train_zip.close()

In [ ]:
# test 데이터 압축 해제
test_path = './Data/test'
if not os.path.exists(test_path):
    test_zip = zipfile.ZipFile('./Data/test.zip')
    test_zip.extractall(test_path)
    test_zip.close()

# 데이터 나누기

In [ ]:
from tqdm import tqdm

In [ ]:
import numpy as np

In [ ]:
# train 데이터 분리
if not os.path.exists('./Data/train_images.npy') or not os.path.exists('./Data/train_data.npy') or not os.path.exists('./Data/train_labels.npy'):
    train_images = []
    train_data = []
    train_labels = []
    for fname in tqdm(os.listdir(train_path)):
        data = np.load(train_path + '/' + fname)

        image = data[:, :, :9]
        train_images.append(image.reshape(40, 40, 9).astype('float32'))

        d = data[:, :, 9:-1]
        train_data.append(d.reshape(40, 40, -1).astype('float32'))

        label = data[:, :, -1]
        label = np.where(label < 0, 0, label)
        train_labels.append(label.reshape(40, 40, 1).astype('float32'))
    np.save('./Data/train_images', train_images)
    np.save('./Data/train_data', train_data)
    np.save('./Data/train_labels', train_labels)

In [ ]:
# test 데이터 분리
if not os.path.exists('./Data/test_images.npy') or not os.path.exists('./Data/test_data.npy'):
    test_images = []
    test_data = []
    for fname in tqdm(os.listdir(test_path)):
        data = np.load(test_path + '/' + fname)

        image = data[:, :, 9]
        test_images.append(image.reshape(40, 40, 9).astype('float32'))

        d = data[:, :, 9:]
        test_data.append(d.reshape(40, 40, -1).astype('float32'))
    np.save('./Data/test_images', test_image)
    np.save('./Data/test_data', test_data)

# 데이터 읽어오기

In [ ]:
# 저장된 train 데이터 읽어오기
train_images = np.load('./Data/train_images.npy')
train_data = np.load('./Data/train_data.npy')
train_labels = np.load('./Data/train_labels.npy')

In [ ]:
# 저장된 test 데이터 읽어오기
test_images = np.load('./Data/test_images.npy')
test_data = np.load('./Data/test_data.npy')

# Tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
# 재생산성을 위해 시드 고정
np.random.seed(7)
tf.set_random_seed(7)

In [ ]:
X = tf.placeholder(tf.float32, [None, 40, 40, 9])
y = tf.placeholder(tf.float32, [None, 40, 40, 1])

In [ ]:
keep_prob = tf.placeholder(tf.float32)

## Layer 1

In [ ]:
W1 = tf.Variable(tf.random_normal([3, 3, 9, 32], stddev = 0.01))
layer1 = tf.nn.relu(tf.nn.conv2d(X, W1, [1, 1, 1, 1], 'SAME'))
layer1 = tf.nn.dropout(tf.nn.max_pool(layer1, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME'), keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## Layer 2

In [ ]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev = 0.01))
layer2 = tf.nn.relu(tf.nn.conv2d(layer1, W2, [1, 1, 1, 1], 'SAME'))
layer2 = tf.nn.dropout(tf.nn.max_pool(layer2, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME'), keep_prob)

## Layer 3

In [ ]:
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev = 0.01))
layer3 = tf.nn.relu(tf.nn.conv2d(layer2, W3, [1, 1, 1, 1], 'SAME'))
layer3 = tf.nn.dropout(tf.nn.max_pool(layer3, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME'), keep_prob)

## Layer 4

In [ ]:
W4 = tf.Variable(tf.random_normal([3, 3, 128, 256], stddev = 0.01))
layer4 = tf.nn.relu(tf.nn.conv2d(layer3, W4, [1, 1, 1, 1], 'SAME'))
layer4 = tf.nn.dropout(tf.nn.max_pool(layer4, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME'), keep_prob)
layer4_flat = tf.reshape(layer4, [-1, 2 * 2 * 256])

## Fully Connected Neural Network

In [ ]:
W5 = tf.get_variable('W5', [2 * 2 * 256, 10], initializer = tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hf = tf.matmul(layer4_flat, W5) + b5

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = hf))

In [ ]:
opt = tf.train.AdamOptimizer(0.01).minimize(cost)

In [ ]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(hf, 1), tf.argmax(y, 1)), dtype = tf.float32))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(20):
        avg_cost = 0
        for i in range(2000):
            _, cost_val, hf_val = sess.run([opt, cost, hf], {X : train_images, y : train_labels, keep_prob : 0.7})
            avg_cost += cost_val / total_batch
        print('epoch : {:04d}, cost : {:.5f}'.format(epoch+1, avg_cost))
    #print('accuracy : {}'.format(sess.run(accuracy, {X : X_test, y : mnist.test.labels, keep_prob : 1.0})))